In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

In [ ]:
# Russell and Nasdaq - 1987-09-10
# S&P 500 1962-01-02

In [ ]:
st_date = "1962-01-02"
ed_date = "2022-01-02"

In [ ]:
ticker = "^GSPC"

In [ ]:
us10 = yf.download("^TNX", start = st_date)
us3mo = yf.download("^IRX", start = st_date)
stocks = yf.download(ticker, start = st_date)
# us10 = yf.download("^TNX", start = st_date, end = ed_date)
# us3mo = yf.download("^IRX", start = st_date, end = ed_date)
# stocks = yf.download("^GSPC", start = st_date, end = ed_date)

In [ ]:
spread = us10 - us3mo
spread = spread.dropna()

In [ ]:
rolling = spread["Adj Close"].rolling(200).mean()

In [ ]:
fig, ax = plt.subplots(figsize = (15,9))
ax.plot(spread["Adj Close"], label = "Spread")
ax.plot(rolling, label = "SMA")
leg = ax.legend();

In [ ]:
# Number of trading days used to calculate returns
# If using the Monte Carlo Options pricer, this number should be the number of trading days
# until the option expires.
n = 25

In [ ]:
# Find log returns of stock market and split bond spread into deciles
log_spy = np.log(stocks.iloc[n:]["Adj Close"].values/stocks.iloc[:-n]["Adj Close"].values)
log_spy = pd.DataFrame(log_spy, index = stocks.index[:-n], columns = ["Log Returns"])
trend_spread = spread["Adj Close"] - rolling
trend_spread = trend_spread.dropna()
binned, bins = pd.qcut(trend_spread, 10, retbins = True)
combined = pd.merge(log_spy, binned, right_index = True, left_index = True)

In [ ]:
# Finding Geometric mean returns of stock market
listed_returns = combined.groupby(["Adj Close"]).agg(lambda x: list(x))
geo_mean = []
for i in listed_returns["Log Returns"]:
    nparray = np.array(i)
    nparray += 1
    gmean = 1
    for j in nparray:
        gmean *= j
    gmean = gmean ** (1/(len(nparray))) - 1
    geo_mean.append(gmean)

In [ ]:
agg_std = combined.groupby(["Adj Close"]).std()
agg_mean = combined.groupby(["Adj Close"]).mean()
agg_median = combined.groupby(["Adj Close"]).median()

In [ ]:
anatable = combined.groupby(["Adj Close"]).describe()

In [ ]:
# anatable.to_csv("anatable.csv")

# Option Pricing Simulator

## Setup

In [ ]:
# Get a sorted list of returns grouped by bond spread

intervals = set(combined["Adj Close"].values)
distr_dict = {}
for interval in intervals:
    distr_dict[interval] = []
    
for i in combined.values:
    temp_list = distr_dict[i[1]]
    temp_list.append(i[0])
    distr_dict[i[1]] = temp_list

for i in distr_dict:
    distr_dict[i].sort()

In [ ]:
# Normalized samples
norm_distr = {}
for i in distr_dict:
    norm_distr[i] = list(np.array(distr_dict[i]) - agg_mean["Log Returns"][i])

In [ ]:
now = trend_spread.iloc[-1]

In [ ]:
for i in intervals:
    if now in i:
        interval = i

In [ ]:
spy_price = yf.download("SPY", start = "2022-06-01")

## Simulation

In [ ]:
# Set to False if you want to incorporate past mean returns into the simulation
# Set to True if you want to assume 0 mean return
normalized = True

In [ ]:
from math import floor, ceil
sample = distr_dict[interval] if normalized == False else norm_distr[interval]
runs = 1000000
np.random.seed(69)
len_sample = len(sample)
sample = [-1] + sample + [1]
sim = []
for i in range(runs):
    rng = np.random.random() * (len_sample+1)
    low_index, high_index = floor(rng), ceil(rng)
    decimal = rng - low_index
    intercept = sample[low_index]
    slope = sample[high_index] - sample[low_index]
    
    sim.append(slope * decimal + intercept)

## Calculator

In [ ]:
strike = 375
option_type = "call"
current_price = spy_price["Adj Close"].iloc[-1]

In [ ]:
if option_type == "put":
    d_sim = strike - np.exp(np.array(sim)) * current_price
    print("Theoretical put price:", np.sum(d_sim[d_sim > 0])/runs)
elif option_type == "call":
    d_sim = np.exp(np.array(sim)) * current_price - strike
    print("Theoretical call price:", np.sum(d_sim[d_sim > 0])/runs)

# Analysis

In [ ]:
print(now)

In [ ]:
anatable

In [ ]:
index = np.arange(len(agg_std)) + 0.3
bar_width = 0.4

fig = plt.figure(figsize = (15, 9))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)

ax = fig.add_subplot(1,1,1)
ax.bar(index, agg_std["Log Returns"], bar_width)
ax.xaxis.set_ticks(range(10))
ax.hlines(anatable["Log Returns"]["std"].iloc[6:].mean(), 6, 10, colors = ["red"], linestyles = "dashed")
ax.hlines(anatable["Log Returns"]["std"].iloc[:6].mean(), 0, 6, colors = ["red"], linestyles = "dashed")
ax.set_title("Effect on 10Y-3M treasury spread trend on S&P 500 volatility",{'fontsize': 19})

plt.show()

In [ ]:
index = np.arange(len(agg_mean)) + 0.3
bar_width = 0.4

fig = plt.figure(figsize = (15, 9))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)

ax = fig.add_subplot(1,1,1)
ax.bar(index, agg_mean["Log Returns"], bar_width)
ax.xaxis.set_ticks(range(10))
ax.hlines(anatable["Log Returns"]["mean"].iloc[6:].mean(), 6, 10, colors = ["red"], linestyles = "dashed")
ax.hlines(anatable["Log Returns"]["mean"].iloc[:6].mean(), 0, 6, colors = ["red"], linestyles = "dashed")
ax.set_title("Effect on 10Y-3M treasury spread trend on S&P 500 mean returns",{'fontsize': 19})

plt.show()

In [ ]:
index = np.arange(len(agg_median)) + 0.3
bar_width = 0.4

fig = plt.figure(figsize = (15, 9))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)

ax = fig.add_subplot(1,1,1)
ax.bar(index, agg_median["Log Returns"], bar_width)
ax.xaxis.set_ticks(range(10))
ax.hlines(anatable["Log Returns"]["50%"].iloc[6:].mean(), 6, 10, colors = ["red"], linestyles = "dashed")
ax.hlines(anatable["Log Returns"]["50%"].iloc[:6].mean(), 0, 6, colors = ["red"], linestyles = "dashed")
ax.set_title("Effect on 10Y-3M treasury spread trend on S&P 500 median returns",{'fontsize': 19})

plt.show()